In [124]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly as py 
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import graphviz
from sklearn import tree
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Importing data

In [125]:
leg_data = pd.read_csv('../input/league-of-legends-game-data/games.csv')
leg_data.head(5)

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,3326086514,1504279457970,1949,9,1,2,1,1,1,1,...,5,0,0,1,1,114,67,43,16,51
1,3229566029,1497848803862,1851,9,1,1,1,1,0,1,...,2,0,0,0,0,11,67,238,51,420
2,3327363504,1504360103310,1493,9,1,2,1,1,1,2,...,2,0,0,1,0,157,238,121,57,28
3,3326856598,1504348503996,1758,9,1,1,1,1,1,1,...,0,0,0,0,0,164,18,141,40,51
4,3330080762,1504554410899,2094,9,1,2,1,1,1,1,...,3,0,0,1,0,86,11,201,122,18


### Data processing 
#### remove seasonld, hero banlist

In [126]:
leg_data.drop(['t2_ban1','t2_ban2','t2_ban3','t2_ban4','t2_ban5'],inplace=True, axis=1)

In [127]:
leg_data.head(5)

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_champ4_sum1,t2_champ4_sum2,t2_champ5id,t2_champ5_sum1,t2_champ5_sum2,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills
0,3326086514,1504279457970,1949,9,1,2,1,1,1,1,...,14,4,412,4,3,5,0,0,1,1
1,3229566029,1497848803862,1851,9,1,1,1,1,0,1,...,4,14,92,4,7,2,0,0,0,0
2,3327363504,1504360103310,1493,9,1,2,1,1,1,2,...,4,11,22,7,4,2,0,0,1,0
3,3326856598,1504348503996,1758,9,1,1,1,1,1,1,...,4,14,22,4,7,0,0,0,0,0
4,3330080762,1504554410899,2094,9,1,2,1,1,1,1,...,4,12,51,4,7,3,0,0,1,0


### Data analyzing
#### Single impact of different resources on winning rate
##### 1. Team side choice
##### 2. game duration
##### 3. First kill
##### 4. inhibitorKills
##### 5. firstBaron
##### 6. dragonKills
##### 7. towerkills


In [128]:
Winning_team = go.Pie(labels=leg_data['winner'].value_counts().values, 
                values=leg_data['winner'].value_counts().values,
                )
               
layout = go.Layout(title='winning condition')

l_data = [Winning_team]
fig = go.Figure(l_data).show()
fig


In [129]:
leg_data['game_duration'] = round(leg_data['gameDuration'] / 60) 

x1 = leg_data[leg_data['winner'] == 1]['game_duration']
x2 = leg_data[leg_data['winner'] == 2]['game_duration'] 

team1_time = go.Histogram(x=x1, bingroup=25, name='team1', opacity=0.9)
team2_time = go.Histogram(x=x2, bingroup=25, name='team2', opacity=0.9)  
bg = go.Layout(title='game_duration') 

data = [trace0, trace1] 
fig = go.Figure(data, bg).show()
fig



In [130]:
def plot_bar_vertical(input_col: str, target_col: str, title_name: str):
    cross_table = round(pd.crosstab(leg_data[input_col], leg_data[target_col], normalize='index')*100, 2)
    index_0 = cross_table.columns.tolist()[0] 
    index_1 = cross_table.columns.tolist()[1] 
    t1 = go.Bar(x=cross_table.index.tolist(),y=cross_table[index_0].values.tolist(),name=index_0,orientation='v', marker=dict(color='rgb(250,520,250)')
                   ) 
    t2 = go.Bar(x=cross_table.index.tolist(), y=cross_table[index_1].values.tolist(), name=index_1,orientation='v',marker=dict(color='rgb(100,300,200)')) 

    data = [t1, t2] 
    layout = go.Layout(title=title_name, bargap=0.4, barmode='stack')

    fig = go.Figure(data=data, layout=layout)
    return fig

In [131]:
plot_bar_vertical('firstBlood', 'winner', 'First blood winning rate').show()  

TypeError: invalid Figure property: bg

In [ ]:
plot_bar_vertical(input_col='t2_inhibitorKills', target_col='winner', title_name='inhibitor related to winning') 

In [ ]:
plot_bar_vertical(input_col='firstBaron', target_col='winner', title_name='First baron r')

In [ ]:
plot_bar_vertical(input_col='t1_dragonKills', target_col='winner', 
                    title_name='T1 dragon related to winning')

In [ ]:
plot_bar_vertical(input_col='t1_towerKills', target_col='winner', title_name='Tower kill realted to winning')  

#### ML training
##### Predict the outcome with multiple important attributes of Machine learning Training
##### 1. remove game with too short game duration
##### 2. Select feature, and importing Decsion Tree.

In [ ]:
leg_data = leg_data[(leg_data['gameDuration'] >= 900)]

In [ ]:
leg_data_model = leg_data[['winner', 'firstBaron', 't1_towerKills', 't1_inhibitorKills','t1_baronKills','t1_dragonKills', 't2_towerKills', 't2_inhibitorKills', 't2_baronKills', 't2_dragonKills'
              ]] 

In [ ]:
x = leg_data_model.drop('winner', axis=1)
y = leg_data_model['winner']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=0)

In [ ]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train,y_train)
print("score:", clf.score(X_test,y_test)) 

In [ ]:
importance_weight = pd.DataFrame(list(zip(X_train.columns, clf.feature_importances_))) 
importance_weight.columns = ['columns', 'importances in model']
importance_weight = importance_weight.sort_values('importances in model', ascending=False)
importance_weight

In [ ]:
importance_weight.plot.bar()